# Análise de Dados do E-commerce Olist com SQL e Python

Olá! meu nome é Matheus Lourenço

Neste projeto, exploro como o SQL pode ser usado de forma eficiente para análise de dados, enquanto o Python entra em cena apenas para visualização e storytelling.

O conjunto de dados utilizado vem da Olist (https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce), uma plataforma brasileira de e-commerce que conecta pequenos lojistas a grandes marketplaces. O dataset inclui mais de 99 mil pedidos realizados entre 2016 e 2018, oferecendo uma base rica para descobrir padrões de vendas, comportamento do cliente e eficiência logística.

Os arquivos CSV originais foram importados para um banco de dados SQLite, onde realizei todo o processo de limpeza, transformação e análise diretamente em SQL. Em seguida, usei Python para criar visualizações que traduzem os resultados em insights visuais e estratégicos.

## Conectando ao Banco de Dados

O SQLite é uma opção bem prática.
Ele vai armazenar todo o banco em um único arquivo e não precisa de servidor ativo em segundo plano.
Usando a biblioteca sqlite3, é possível acessar e consultar os dados diretamente pelo Python.

In [ ]:
import pandas as pd
import sqlite3
import os

# Conectar/criar o banco
conn = sqlite3.connect('/content/olist.sqlite')

# Lista de CSVs e nomes das tabelas
csv_files = {
    "customers": "olist_customers_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "orders": "olist_orders_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "category_translation": "product_category_name_translation.csv"
}

# Carregando cada CSV e salvar no banco SQLite
for table_name, file_name in csv_files.items():
    df = pd.read_csv(f"/content/{file_name}")
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Tabela '{table_name}' criada com {len(df)} registros.")

# Confirmando as tabelas criadas
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("\nTabelas disponíveis no banco:")
print(tables)



Para consultar o banco de dados, basta escrever a query SQL como uma string e passá-la para o 'pandas.read_sql_query', junto com a conexão do banco.
O resultado é retornado em um DataFrame do pandas, facilitando a análise e visualização dos dados.

Ao longo deste notebook, utilizaremos a seguinte função para explorar as tabelas do banco de dados de forma simples e direta:

In [ ]:
import pandas as pd

def view_table(table, limit=5):
    query = f"""
        SELECT *
        FROM {table}
        LIMIT {limit}
    """
    return pd.read_sql_query(query, conn)

# visualizar as 5 primeiras linhas da tabela 'orders'
view_table('orders')

# Número de Pedidos
A tabela 'orders' reúne os IDs de pedidos e clientes, além do status de entrega, que pode ser “delivered” ou outros estados anteriores à entrega.

In [ ]:
# Tabela de pedidos, mostrando as 3 primeiras colunas
view_table('orders', 5).iloc[:, :3]

As próximas colunas mostram os horários das mudanças de status do pedido e, por último, a data estimada de entrega

In [ ]:
# resto das colunas
view_table('orders', 5).iloc[:, 3:]

In [ ]:
# Contagem de pedidos por dia
orders_per_day = """
SELECT
    DATE(order_purchase_timestamp) AS day,
    COUNT(*) AS order_count
FROM orders
GROUP BY day
ORDER BY day
"""

df = pd.read_sql_query(orders_per_day, conn)
df.head(5)

No resultado anterior, podemos ver que os primeiros dias possuem poucos pedidos. Vamos usar o Matplotlib para visualizar a frequência de pedidos por dia:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Aumento do tamanho da fonte
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16

# Plotando o gráfico de linha
plt.figure(figsize=(14, 6))
plt.plot(pd.to_datetime(df['day']), df['order_count'], color='green')
plt.ylabel('Número de pedidos')
plt.title('Número de pedidos por dia')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.xticks(rotation=45)
plt.show()

Pode-se perceber um pico de pedidos próximo ao Natal, especialmente em 24 de dezembro. Também há um crescimento constante nas vendas ao longo do tempo, refletindo a expansão da Olist. Como o início e o fim do período têm poucos dados, essas datas serão ignoradas nas próximas análises.

Agora, vamos investigar quando os pedidos acontecem — por dia da semana e hora do dia — criando um mapa de calor que mostra o volume de pedidos ao longo da semana e das horas. Para isso, extrairemos o dia da semana e a hora do pedido usando a função STRFTIME.

In [ ]:
order_day_hour = """
SELECT
    -- dia da semana abreviado
    CASE STRFTIME('%w', order_purchase_timestamp)
        WHEN '1' THEN 'Mon'
        WHEN '2' THEN 'Tue'
        WHEN '3' THEN 'Wed'
        WHEN '4' THEN 'Thu'
        WHEN '5' THEN 'Fri'
        WHEN '6' THEN 'Sat'
        WHEN '0' THEN 'Sun'
        END AS day_of_week_name,
    -- dia da semana como um inteiro (Sunday=7)
    CAST(STRFTIME('%w', order_purchase_timestamp) AS INTEGER) AS day_of_week_int,
    -- Hour of the day (0-24)
    CAST(STRFTIME("%H", order_purchase_timestamp) AS INTEGER) AS hour
FROM orders
"""

pd.read_sql_query(order_day_hour, conn)

Usamos uma 'list comprehension' em Python para gerar automaticamente a contagem de pedidos por hora, sem precisar escrever 24 linhas manuais. A consulta anterior é reaproveitada como uma subconsulta temporária para facilitar a análise.

In [ ]:
count_orders_per_hour = ',\n    '.join([
    f'COUNT(CASE WHEN hour = {i} THEN 1 END) AS "{i}"' \
    for i in range(24)
])

orders_per_day_of_the_week_and_hour = f"""
WITH OrderDayHour AS (
    {order_day_hour}
)
SELECT
    day_of_week_name,
    {count_orders_per_hour}
FROM OrderDayHour
GROUP BY day_of_week_int
ORDER BY day_of_week_int
"""

In [ ]:
# SQL query sem uma subconsulta temporária
print(orders_per_day_of_the_week_and_hour[591:])

Depois de executar a consulta, precisamos apenas definir a coluna 'day_of_the_week_name' como o índice do dataframe para obter a matriz necessária para construir o mapa de calor:

In [ ]:
df = pd.read_sql_query(orders_per_day_of_the_week_and_hour, conn)
df = df.set_index('day_of_week_name')
df

Podemos usar a biblioteca do python 'seaborn' para criar o mapa de calor:

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(14, 6))
sns.heatmap(df, cmap='YlGnBu', cbar=False)
media_orders = df.mean().mean()
for i in range(len(df)):
    for j in range(len(df.columns)):
        text_color = 'white' if df.iloc[i, j] > media_orders else 'black'
        ax.text(j+0.5, i+0.5, int(df.iloc[i, j]),
            color=text_color, fontsize=10, ha="center", va="center")
plt.title("Número de dias por dia da semana e pela hora do dia")
plt.xlabel("Hora do dia")
plt.ylabel("")
plt.show()

Pode-se observar que a maioria dos pedidos foi feita em dias de semana, entre *10h e 16h*, com uma leve queda por volta do meio-dia (horário de almoço). Há também um aumento de pedidos à noite, por volta das 21h, de domingo a quinta-feira.

O sábado é o dia com menos pedidos, embora ainda movimentado.
Já o horário com menor atividade é entre 3h e 5h da manhã, mas ainda ocorrem algumas compras nesse período

Agora que analisamos a distribuição dos pedidos ao longo do tempo, vamos observar a distribuição geográfica.
A tabela 'customers' contém o prefixo do CEP, cidade e estado de cada cliente.
Vale notar que existem dois identificadores: 'customer_id '(relacionado ao pedido) e 'customer_unique_id', que representa o cliente único.

In [ ]:
view_table('customers', 5)

## Quais são as cidades com maior número de pedidos no conjunto de dados?


Vamos criar uma query que retorna o top 10 dessas cidades. Para isso, faremos um JOIN entre as tabelas 'orders' e 'customers'.

In [ ]:
orders_per_city = """
SELECT
    customer_city AS customer_city,
    UPPER(customer_city) AS city,
    COUNT(orders.order_id) AS city_order_count
FROM
    customers
    JOIN orders USING (customer_id)
GROUP BY customer_city
ORDER BY city_order_count DESC
LIMIT 10
"""

pd.read_sql_query(orders_per_city, conn)

Vamos visualizar essas cidades em um gráfico de barras horizontais com o 'kind=barh'.
Como a função barh do Matplotlib plota de baixo para cima, precisamos inverter a ordem dos resultados. Para isso, usaremos a query anterior como uma subquery no SQL.

In [ ]:
orders_per_city_reversed = f"""
SELECT *
FROM ({orders_per_city})
ORDER BY city_order_count
"""

In [ ]:
top10_cidades = pd.read_sql_query(orders_per_city_reversed, conn)

plt.figure(figsize=(10, 6))
plt.barh(top10_cidades['city'], top10_cidades['city_order_count'])
plt.xlabel('Número de pedidos')
plt.ylabel('Cidade')
plt.title('Top 10 cidades por número de pedidos')
plt.show()

Podemos ver que São Paulo domina amplamente, com aproximadamente o dobro de pedidos em relação ao segundo lugar, Rio de Janeiro.
As demais cidades — Belo Horizonte, Brasília, Curitiba e Campinas — têm volumes consideravelmente menores, mas ainda expressivos.
O padrão indica forte concentração de pedidos nas grandes capitais e regiões metropolitanas, o que reflete maior densidade populacional e poder de compra

<br>

# Preço dos pedidos

- Qual é o valor médio de um pedido?
- A Olist lida com transações de itens de valor muito baixo ou muito alto?

vamos dar uma olhada na tabela order_items:
Ela contém vários IDs, além do preço (em reais R$) e do custo de envio dos itens de cada pedido

In [ ]:
# Tabela order_items, primeiras 4 colunas
view_table('order_items', 5).iloc[:, :4]

In [ ]:
# Tabela order_items, 2 últimas colunas
view_table('order_items', 5).iloc[:, 5:]

Qual a média do preço dos pedidos, considerando os custos de produtos e o frete pra envio? vamos analisar os pedidos com menores e maiores custos

In [ ]:
order_price_stats = """
SELECT
    MIN(order_price) AS min_order_price,
    ROUND(AVG(order_price), 2) AS avg_order_price,
    MAX(order_price) AS max_order_price
FROM (
    SELECT
        orders.order_id,
        SUM(order_items.price + order_items.freight_value) AS order_price
    FROM orders
        JOIN order_items USING (order_id)
    GROUP BY orders.order_id
)
"""

pd.read_sql_query(order_price_stats, conn)

O preço médio dos pedidos é R$160,58

O pedido mais caro chega a R$13.664,08, quase 100x acima da média — provavelmente uma distribuição com cauda à direita

Vamos ver a variação dos preços, separando o valor dos produtos e o frete.

In [ ]:
order_product_and_shipping_costs = """
SELECT
    orders.order_id,
    SUM(price) AS product_cost,
    SUM(freight_value) AS shipping_cost
FROM
    orders
    JOIN order_items USING (order_id)
WHERE order_status = 'delivered'
GROUP BY orders.order_id
"""

df = pd.read_sql_query(order_product_and_shipping_costs, conn)
df

Vamos plotar um histograma para cada tipo de custo.

Como os valores variam bastante, mas a maioria dos pedidos tem custo baixo, vamos limitar o eixo X para 500 reais (produtos) e 80 reais (frete), podendo ver a distribuição dos valores mais comuns.

In [ ]:
plt.figure(figsize=(15, 6))
# Histograma para o custo total de produtos
plt.subplot(1, 2, 1)
plt.hist(df['product_cost'], bins=1000, color='blue')
plt.title('Custo do produto para pedidos < R$500')
plt.xlabel('Custo do produto (R$)')
plt.ylabel('Frequência')
plt.xlim([0, 500])
# Histograma para o custo total do frete
plt.subplot(1, 2, 2)
plt.hist(df['shipping_cost'], bins=800, color='#ad865f')
plt.title('Custo do frete para pedidos < R$80')
plt.xlabel('Custo do produto (R$)')
plt.xlim([0, 80])
plt.show()

O valor dos produtos varia bastante, mas a maioria dos pedidos custa menos de R$ 200.

O frete geralmente fica entre 7 a 20 reais, podendo chegar a valores bem mais altos em alguns casos.

# Categorias de produtos

vamos observar a tabela 'products', que possui 9 colunas. Vamos analistar a categoria e o peso do produto

In [ ]:
view_table('products', 5).iloc[:, [0, 1, 5]]

In [ ]:
query = """
SELECT DISTINCT product_category_name
FROM products
ORDER BY product_category_name;
"""

df = pd.read_sql_query(query, conn)
print(df)


São 73 categorias únicas de produtos. Vamos usar um treemap para mostrar as vendas relativas de cada uma. Como 71 é muito, selecionaremos as 18 principais e agruparemos o resto em “Outras categorias”.

Primeiro, vamos calcular o total de vendas por categoria

In [ ]:
ranked_categories = """
SELECT
    product_category_name AS categoria,
    SUM(price) AS vendas,
    RANK() OVER (ORDER BY SUM(price) DESC) AS posicao
FROM order_items
    JOIN orders USING (order_id)
    JOIN products USING (product_id)
WHERE order_status = 'delivered'
GROUP BY product_category_name
"""

pd.read_sql_query(ranked_categories, conn)

In [ ]:
category_sales_summary = f"""
WITH RankedCategories AS (
    {ranked_categories}
)
-- Top 18 categorias por vendas
SELECT
    categoria,
    vendas
FROM RankedCategories
WHERE posicao <= 18

UNION ALL

-- Outras categorias agrupadas
SELECT
    'Outras categorias' AS categoria,
    SUM(vendas) AS vendas
FROM RankedCategories
WHERE posicao > 18
"""

df = pd.read_sql_query(category_sales_summary, conn)
df

Vamos visualizar esses dados em um 'treemap' usando o a biblioteca 'Squarify':

In [ ]:
!pip install squarify

In [ ]:
import squarify

plt.figure(figsize=(15, 8))
plt.title('Vendas por categoria')
color = sns.color_palette("viridis", len(df))
squarify.plot(sizes=df['vendas'], label=df['categoria'],
              alpha=0.7, color=color, edgecolor="white", linewidth=2)
plt.axis('off')
plt.show()

Podemos entender melhor os tipos de produtos analisando a distribuição do peso por categoria em boxplots.
Usaremos a lista das 18 principais categorias do dataframe anterior.

In [ ]:
top_18_categorias = tuple(categoria for categoria in df['categoria'] if categoria != 'Outras categorias')
print(top_18_categorias)

Como o SQLite não possui função de mediana, ordenaremos os produtos por categoria usando números de linha e contaremos quantos produtos há em cada uma, salvando o resultado para gerar o gráfico.

In [ ]:
categorias_ordenadas = f"""
SELECT
    product_weight_g AS peso,
    product_category_name AS categoria,
    ROW_NUMBER() OVER(PARTITION BY product_category_name ORDER BY product_weight_g) AS categoria_linha,
    COUNT(*) OVER(PARTITION BY product_category_name) AS total_produtos
FROM products
JOIN order_items USING (product_id)
WHERE product_category_name IN {top_18_categorias}
"""

df = pd.read_sql_query(categorias_ordenadas, conn)
df

Podemos usar a consulta anterior 'categorias_ordenadas' para calcular a mediana de cada categoria e ordenar pelos valores medianos.

In [ ]:
categorias_por_mediana = f"""
WITH CategoriasOrdenadas AS (
    {categorias_ordenadas}
)
SELECT
    categoria,
    AVG(peso) AS mediana_peso
FROM CategoriasOrdenadas
WHERE
    -- Número ímpar de produtos: seleciona a linha do meio
    (total_produtos % 2 = 1 AND categoria_linha = (total_produtos + 1) / 2)
    OR
    -- Número par de produtos: seleciona as duas linhas centrais e calcula a média
    (total_produtos % 2 = 0 AND categoria_linha IN ((total_produtos / 2), (total_produtos / 2 + 1)))
GROUP BY categoria
ORDER BY mediana_peso;
"""

categorias_por_mediana_df = pd.read_sql_query(categorias_por_mediana, conn)
categorias_por_mediana_df



Agora bora plotar um gráfico com os boxplots.

In [ ]:
plt.figure(figsize=(12, 8))

# Define a ordem das categorias de acordo com a mediana calculada
ordem = categorias_por_mediana_df['categoria'].tolist()

# Cria o boxplot de peso por categoria
sns.boxplot(
    x='peso',
    y='categoria',
    data=df,
    order=ordem,
    showfliers=False
)

# Customização dos eixos e título
plt.xlabel('Peso do produto (gramas)')
plt.ylabel('Categoria do produto')
plt.title('Distribuição do peso dos produtos - Top 18 categorias por vendas')

# Limites e ticks do eixo X
plt.xlim(-100, 26100)
plt.xticks(ticks=range(0, 30000, 2500))
plt.yticks(fontsize=12)

plt.show()


Podemos ver no topo do gráfico que as categorias telefonia, informática_acessórios, relógios_presentes e beleza_saúde possuem itens leves (geralmente abaixo de 2 kg).
Na parte inferior, pcs e moveis_escritório têm produtos bem mais pesados.

# Evolução mensal das vendas por categoria

Vamos analisar as tendências de vendas de algumas categorias selecionadas:

In [ ]:
selected_categories = ('beleza_saude',
    'automotivo',
    'brinquedos',
    'informatica_acessorios',
    'cama_mesa_banho')

Para criar um gráfico de linha com as vendas mensais de cada categoria, precisamos de uma matriz com a soma das vendas, onde as colunas são as categorias e as linhas representam os meses.

In [ ]:
monthly_sales_selected_categories = f"""
SELECT
    strftime('%Y-%m', order_purchase_timestamp) AS ano_mes,
    SUM(CASE WHEN product_category_name = 'beleza_saude' THEN price END) AS beleza_saude,
    SUM(CASE WHEN product_category_name = 'automotivo' THEN price END) AS automotivo,
    SUM(CASE WHEN product_category_name = 'brinquedos' THEN price END) AS brinquedos,
    SUM(CASE WHEN product_category_name = 'informatica_acessorios' THEN price END) AS informatica_acessorios,
    SUM(CASE WHEN product_category_name = 'cama_mesa_banho' THEN price END) AS cama_mesa_banho
FROM orders
    JOIN order_items USING (order_id)
    JOIN products USING (product_id)
WHERE order_purchase_timestamp >= '2017-01-01'
    AND product_category_name IN {selected_categories}
GROUP BY ano_mes
"""

df = pd.read_sql_query(monthly_sales_selected_categories, conn)
df = df.set_index('ano_mes')
df


Há poucos dados antes de 2017-01-01, então ignorei os pedidos feitos antes dessa data. Visualizando a série temporal:

In [ ]:
# Converte o índice (ano_mes) para datetime
df.index = pd.to_datetime(df.index, format='%Y-%m')

# Cria figura e eixo
fig, ax = plt.subplots(figsize=(14, 8))

# Plota as linhas
df.plot(ax=ax, marker='o', linestyle='-')

# Ajusta os rótulos do eixo X
ax.set_xticks(df.index)
ax.set_xticklabels(df.index.strftime('%Y-%m'), rotation=45, ha='right')

# Títulos e rótulos
plt.title('Evolução Mensal das Vendas por Categoria', fontsize=16)
plt.xlabel('Ano-Mês', fontsize=12)
plt.ylabel('Vendas Mensais (R$)', fontsize=12)

# Legenda e grid
plt.legend(title='Categoria de Produto', title_fontsize=13, fontsize=11)
plt.grid(True, linestyle='--', alpha=0.5)

# Mostra o gráfico
plt.tight_layout()
plt.show()


Beleza e Saúde lidera com crescimento constante e sólido; Automotivo cresce de forma estável e previsível.
Brinquedos tem pico forte no fim do ano, mostrando alta sazonalidade.
Informática sobe até 2018 e depois cai, exigindo revisão de oferta.
Cama, Mesa e Banho oscila bastante, com potencial em datas específicas.

# Entrega dos pedidos

A tabela orders contém vários registros de tempo:

- order_purchase_timestamp – quando o cliente faz o pedido.
- order_approved_at – quando o pedido é aprovado pela Olist.
- order_delivered_carrier_date – quando é enviado para a transportadora.
- order_delivered_customer_date – quando o cliente recebe o pedido.
- order_estimated_delivery_date – previsão de entrega.

Cada um marca uma etapa do processo de envio.
Agora vamos consultar os dados para visualizar o tempo médio de cada etapa nas 10 cidades com mais pedidos.

In [ ]:
# Top 10 cidades com mais pedidos
top_cities_query = """
SELECT
    customer_city,
    COUNT(order_id) AS total_pedidos
FROM orders
JOIN customers USING (customer_id)
GROUP BY customer_city
ORDER BY total_pedidos DESC
LIMIT 10
"""

top_cities = pd.read_sql_query(top_cities_query, conn)
top_cities

In [ ]:
order_stage_times_top_10_cities = f"""
SELECT
    UPPER(customer_city) AS city,
    AVG(JULIANDAY(order_approved_at) - JULIANDAY(order_purchase_timestamp)) AS approved,
    AVG(JULIANDAY(order_delivered_carrier_date) - JULIANDAY(order_approved_at)) AS delivered_to_carrier,
    AVG(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_delivered_carrier_date)) AS delivered_to_customer,
    AVG(JULIANDAY(order_estimated_delivery_date) - JULIANDAY(order_delivered_customer_date)) AS estimated_delivery
FROM orders
JOIN customers USING (customer_id)
WHERE customer_city IN {tuple(top_cities['customer_city'])}
GROUP BY city
ORDER BY approved + delivered_to_carrier + delivered_to_customer DESC
"""

df = pd.read_sql_query(order_stage_times_top_10_cities, conn)
df = df.set_index('city')
df

Bora ver em um gráfico de barra o nosso df

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
df.plot(kind='barh', stacked=True, color=['#c44f53', '#4c72b1', '#55a978', '#dd842275'], ax=ax)
ax.set_xlabel('Média de dias')
ax.set_ylabel('Cidade')
fig.suptitle('Média de dias em cada etapa do pedido (10 principais cidades por vendas)', fontsize=16, x=0.38, y=0.92)
ax.grid(True, linestyle='--', linewidth=0.5, axis='x')
max_bar_length = int(df.sum(axis=1).max())
ax.set_xticks(range(0, max_bar_length + 4))
ax.tick_params(axis='y', labelsize=14)
plt.legend(title='Etapa do pedido', title_fontsize=14, fontsize=14)
plt.show()

Vemos que o tempo entre a aprovação e o envio ao transportador é parecido em todas as cidades (aproximadamente 3 dias).
Já o tempo até a entrega varia bastante: São Paulo, Guarulhos e São Bernardo levam cerca de 5 dias, enquanto Rio, Porto Alegre e Salvador demoram mais que o dobro.

As estimativas de entrega são, em média, uma semana mais longas que o real, mostrando previsões conservadoras.

Agora, vamos analisar se há variação sazonal nos prazos de entrega:

In [ ]:
#vamos criar uma query para calcular o tempo médio de entrega por DIA
daily_avg_shipping_time = """
SELECT
    -- Cpmvertemos o timestamp da compra em apenas data (sem hora)
    DATE(order_purchase_timestamp) AS purchase_date,
    -- Calculamos o tempo médio (em dias) entre a compra e a entrega ao cliente
    AVG(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp))
        AS avg_delivery_time
FROM orders
-- Usamos apenas os pedidos dentro do período de 1 ano (jun/2017 a jun/2018)
WHERE order_purchase_timestamp >= '2017-06-01' AND order_purchase_timestamp <= '2018-06-30'
-- Agrupamos por data de compra para obter a média diária
GROUP BY DATE(order_purchase_timestamp)
"""

df = pd.read_sql_query(daily_avg_shipping_time, conn)
df

Plotando um gráfico de linha, conseguimos colocar uma linha em vermelho pra nos mostrar o tempo médio global de entrega durante o período.

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(pd.to_datetime(df['purchase_date']), df['avg_delivery_time'], label='Média diária', color='green')
plt.axhline(y=df['avg_delivery_time'].mean(), color='red', linestyle='--', label='Média anual')
plt.ylabel('Dias')
plt.title('Tempo médio de entrega (de Junho 2017 até Junho 2018)')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.xticks(rotation=45)
plt.legend()
plt.show()

Como podemos ver, há dois períodos em que o tempo de entrega foi significativamente maior que a média: dezembro de 2017 e fevereiro–março de 2018.

Os atrasos em dezembro provavelmente foram causados pelo movimento intenso das festas de fim de ano, enquanto os de fevereiro–março podem ter sido resultado das [greves](https://agenciabrasil.ebc.com.br/geral/noticia/2018-03/contra-mudancas-no-plano-de-saude-trabalhadores-dos-correios-fazem-ato-no-rio) dos correios que ocorreram em alguns estados do Brasil nesse período.

# Avaliações de pedidos

O banco de dados possui uma tabela de avaliações, onde os clientes atribuem notas de 1 a 5 e podem deixar comentários. Vamos ver as colunas usadas na análise.

In [ ]:
view_table('order_reviews', 5).iloc[:, [1, 2, 4]]

Vamos contar quantos pedidos tem para cada nota de aval:

In [ ]:
review_score_count = """
SELECT
    review_score,
    COUNT(*) AS count
FROM order_reviews
GROUP BY review_score
"""
df = pd.read_sql_query(review_score_count, conn)
df

In [ ]:
plt.figure(figsize=(10,6))
colors = ['#BC2023', '#EB442C', '#F8B324', '#6da814', '#0C6B37']
sns.barplot(x='review_score', y='count', data=df, hue='review_score', palette=colors, dodge=False)
plt.title('Distribuição das notas')
plt.xlabel('Nota de avaliação')
plt.ylabel('Quantidade')
plt.legend().remove()
plt.tight_layout()
plt.show()


Podemos observar que a maioria das avaliações são positivas, mas ainda há um número considerável de clientes insatisfeitos.

Para entender o motivo, vamos gerar uma nuvem de palavras com os comentários que receberam nota 1 ou 2, unindo todas as mensagens com a função GROUP_CONCAT do SQL

In [ ]:
comments_negativos = """
SELECT GROUP_CONCAT(review_comment_message, ' ') AS comments
FROM order_reviews
WHERE review_score IN (1,2)
"""

comments_negativos_df = pd.read_sql(comments_negativos, conn)['comments'][0]
comments_negativos_df[:150]

In [ ]:
## gerando a nuvem de palavras
from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(comments_negativos_df)
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

A principal causa dos comentários negativos está relacionada a atrasos na entrega, enquanto a insatisfação com a qualidade do produto ou erros no pedido aparentam ser fatores secundários.

# Valor do tempo de vida do cliente

Valor do tempo de vida do cliente (LTV) é uma métrica que estima o valor total que um cliente trará ao longo do relacionamento com a empresa. Vamos calcular o LTV usando a tabela 'order_payments', que mostra os pagamentos e métodos utilizados.

In [ ]:
view_table('order_payments', 5)

Para começar, vamos calcular os componentes do LTV para cada cliente:

Frequência de Compra (FC): número de pedidos feitos pelo cliente.

Valor Médio do Pedido (VMP): total pago dividido pelo número de pedidos.

Tempo Médio de Vida do Cliente (TMVC): número de semanas entre o primeiro e o último pedido (mínimo de 1).

Para deixar o código mais claro, usaremos uma tabela comum para reunir os dados necessários e, em seguida, calcular cada componente do LTV na consulta principal.

In [ ]:
cte = """
-- Criamos uma CTE (subquery nomeada) chamada CustomerData
-- Ela reúne os dados principais de cada cliente (pedidos, pagamentos e datas)
WITH CustomerData AS (
    SELECT
        customer_unique_id,
        customer_zip_code_prefix AS zip_code_prefix,
        COUNT(DISTINCT orders.order_id) AS order_count,
        SUM(payment_value) AS total_payment,
        JULIANDAY(MIN(order_purchase_timestamp)) AS first_order_day,
        JULIANDAY(MAX(order_purchase_timestamp)) AS last_order_day
    FROM customers
        JOIN orders USING (customer_id)
        JOIN order_payments USING (order_id)
    GROUP BY customer_unique_id
)

-- A partir da CTE, calculamos os indicadores de valor do cliente (CLV)
SELECT
    customer_unique_id,
    zip_code_prefix,
    order_count AS FC,
    total_payment / order_count AS VMP,
    CASE
     -- Se a diferença entre o primeiro e último pedido for menor que 7 dias, define o tempo mínimo como 1 semana
        WHEN (last_order_day - first_order_day) < 7 THEN 1
            -- Caso contrário, calcula o tempo médio de vida do cliente em semanas
        ELSE
            (last_order_day - first_order_day) / 7
        END AS TMVC -- Tempo médio de vida do cliente
FROM CustomerData
"""

pd.read_sql(cte, conn)

Para criar um mapa da distribuição do LTV pelo Brasil, também precisamos adicionar a latitude e a longitude de cada prefixo do CEP, que podem ser encontradas na tabela 'geolocation'.

In [ ]:
view_table('geolocation', 5)

sabe-se que CLV = FC × VMP × TMVC
onde:

FC = Frequência de compras (quantas vezes o cliente comprou)

VMP = Valor médio por pedido

TMVC = Tempo médio de vida do cliente

In [ ]:
clv_por_cep = f"""
WITH CustomerData AS (
    SELECT
        customer_unique_id,
        customer_zip_code_prefix AS zip_code_prefix,
        COUNT(DISTINCT orders.order_id) AS FP, -- Frequência de compras
        SUM(payment_value) / COUNT(DISTINCT orders.order_id) AS VMP, -- Valor médio por pedido
        CASE
            WHEN (JULIANDAY(MAX(order_purchase_timestamp)) - JULIANDAY(MIN(order_purchase_timestamp))) < 7 THEN 1
            ELSE (JULIANDAY(MAX(order_purchase_timestamp)) - JULIANDAY(MIN(order_purchase_timestamp))) / 7
        END AS TMVC -- Tempo médio de vida do cliente (em semanas)
    FROM customers
        JOIN orders USING (customer_id)
        JOIN order_payments USING (order_id)
    GROUP BY customer_unique_id
),
CLVPorCEP AS (
    SELECT
        zip_code_prefix,
        AVG(FP * VMP * TMVC) AS CLV_medio,
        COUNT(*) AS numero_clientes
    FROM CustomerData
    GROUP BY zip_code_prefix
)
SELECT
    c.zip_code_prefix,
    c.CLV_medio,
    c.numero_clientes,
    g.geolocation_lat AS latitude,
    g.geolocation_lng AS longitude
FROM CLVPorCEP c
JOIN geolocation g
    ON c.zip_code_prefix = g.geolocation_zip_code_prefix
GROUP BY c.zip_code_prefix
"""

In [ ]:
df = pd.read_sql(clv_por_cep, conn)
print(df.columns)

Podemos criar um mapa usando a biblioteca folium do python!
https://realpython.com/python-folium-web-maps-from-data/

In [ ]:
import folium
import numpy as np

mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

for i, linha in df.iterrows():
    folium.CircleMarker(
        location=[linha['latitude'], linha['longitude']],
        radius=0.1 * np.sqrt(linha['numero_clientes']),
        color=None,
        fill_color='#85001d',
        fill_opacity=0.1 + 0.1 * np.sqrt(linha['CLV_medio'] / df['CLV_medio'].max()),
        popup=(
            f"<b>Prefixo CEP:</b> {int(linha['zip_code_prefix'])}<br>"
            f"<b>CLV Médio:</b> {linha['CLV_medio']:.2f}<br>"
            f"<b>Clientes:</b> {int(linha['numero_clientes'])}"
        )
    ).add_to(mapa)

mapa



Como podemos ver no mapa, a maior parte do valor dos clientes dos vendedores da Olist está concentrada no sudeste do Brasil, nas regiões mais populosas — principalmente no estado de São Paulo e nas áreas próximas às cidades do Rio de Janeiro e Belo Horizonte.

<br>

# Vendedores

vamos ver as primeiras linhas da tabela 'sellers'

In [ ]:
view_table('sellers', 5)

como os vendedores são em sua maioria?
pequenos com poucas vendas ou grandes com alto volume? E como as avaliações variam entre eles?
Podemos responder a isso com um gráfico de dispersão talvez, usando dados da tabela 'sellers'.

In [ ]:
# Consulta SQL para analisar a relação entre volume de vendas e pontuação média de avaliações dos vendedores
seller_review_scores_and_sales = '''
SELECT
    sellers.seller_id,  -- Identificador único de cada vendedor
    AVG(order_reviews.review_score) AS avg_review_score,  -- Média das notas de avaliação recebidas
    SUM(order_items.price) AS total_sales,                -- Soma total das vendas realizadas (receita bruta)
    COUNT(orders.order_id) AS num_orders                  -- Quantidade de pedidos feitos ao vendedor
FROM
    sellers
    -- Associa os itens de pedido para saber o que cada vendedor vendeu
    LEFT JOIN order_items ON sellers.seller_id = order_items.seller_id
    -- Conecta os pedidos para relacionar vendas e avaliações
    LEFT JOIN orders ON order_items.order_id = orders.order_id
    -- Junta as avaliações dos clientes aos pedidos
    LEFT JOIN order_reviews ON orders.order_id = order_reviews.order_id
GROUP BY
    sellers.seller_id  -- Agrupa as métricas por vendedor
HAVING
    COUNT(orders.order_id) > 10  -- Mantém apenas vendedores com mais de 10 pedidos (filtra outliers e dados irrelevantes)
'''

df = pd.read_sql_query(seller_review_scores_and_sales, conn)
df

Podemos criar um gráfico de dispersão onde cada ponto representa um vendedor:
no eixo Y estará a média das avaliações dos pedidos, no eixo X, o total de vendas, exibido em escala logarítmica (já que a maioria vende pouco). Além disso, a cor e o tamanho dos pontos indicarão a quantidade de pedidos de cada vendedor.

In [ ]:
plt.figure(figsize=(15, 8))
sns.scatterplot(data=df, x='total_sales', y='avg_review_score', size='num_orders', sizes=(10, 500),
                hue='num_orders', palette="crest", alpha=0.7)
plt.xscale('log')
plt.xlabel('Total de vendas (escala logarítmica)')
plt.ylabel('Média das notas de avaliação')
plt.title('Vendedores: Notas de avaliação x vendas')
plt.legend(title='Número de pedidos')
plt.show()

A maioria dos vendedores da Olist é pequena, com poucas vendas.
Vendedores maiores têm avaliações mais estáveis, geralmente entre 3.5 e 4.5.
À direita do gráfico, destaca-se um grande vendedor (coloração mais roxa) com notas bem abaixo dos demais.

Também tem casos de vendedores com o mesmo total de vendas, mas com diferentes números de pedidos — indicando valores médios por pedido distintos.

Vamos tentar criar outro gráfico para entender melhor como os vendedores se distribuem pelo volume de pedidos.

Para isso, vamos agrupá-los em 4 categorias com base na quantidade total de pedidos enviados:

- Grupo 1: 1 a 9 pedidos
- Grupo 2: 10 a 99 pedidos
- Grupo 3: 100 a 999 pedidos
- Grupo 4: 1000 ou mais pedidos

In [ ]:
vendedores_segmentados = """
SELECT
    seller_id,
    CASE
        WHEN COUNT(order_id) BETWEEN 1 AND 9 THEN '1-9 orders'
        WHEN COUNT(order_id) BETWEEN 10 AND 99 THEN '10-99 orders'
        WHEN COUNT(order_id) BETWEEN 100 AND 999 THEN '100-999 orders'
        ELSE '1000+ orders'
    END AS bucket
FROM order_items
GROUP BY seller_id
"""

pd.read_sql_query(vendedores_segmentados, conn).head(5)

In [ ]:
vendedores_por_segmento = f"""
WITH VendedoresSegmentados AS (
    {vendedores_segmentados}
)
SELECT
    bucket,
    COUNT(seller_id) AS seller_count
FROM VendedoresSegmentados
GROUP BY bucket
"""

seller_buckets = pd.read_sql_query(vendedores_por_segmento, conn)
seller_buckets


Vamos visualizar o df.

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='bucket', y='seller_count', data=seller_buckets, hue='bucket', palette="magma", dodge=False)
plt.title('Número de vendedores por pedidos (agrupados)')
plt.xlabel('Quantidade de pedidos por vendedor')
plt.ylabel('Número de vendedores')
plt.tight_layout()
plt.show()

Pode-se concluir que a maioria dos vendedores apresentam poucos pedidos (1 a 9 pedidos), e poucos vendedores ultrampassam 1000 pedidos, que é um valor alto de pedidos.


- Os grandes vendedores têm tempos de envio menores?
- Para responder a essa pergunta, vamos tentar criar um gráfico com a distribuição do tempo de envio para cada um dos quatro grupos anteriores.
- Vamos reutilizar a tabela anterior na consulta para obter os dados necessários.

In [ ]:
tempos_envio_vendedores = f"""
-- Criamos uma CTE (tabela temporária) chamada 'VendedoresSegmentados'
-- que reutiliza a query anterior, onde os vendedores foram agrupados
-- em faixas (1-9 pedidos, 10-99, 100-999, 1000+)
WITH VendedoresSegmentados AS (
    {vendedores_segmentados}
)

-- Agora selecionamos os dados principais que queremos analisar

SELECT
    bucket,
    VendedoresSegmentados.seller_id,
    -- Calculamos o tempo de entrega em dias:
    -- diferença entre a data de entrega ao cliente e a data da compra
    JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp) AS delivery_time
FROM orders
    -- Relacionamos a tabela de pedidos com os itens de pedido
    JOIN order_items USING (order_id)
    -- Ligamos também à tabela 'VendedoresSegmentados' para trazer o grupo de cada vendedor
    JOIN VendedoresSegmentados USING (seller_id)
WHERE order_status = 'delivered'
"""

df = pd.read_sql_query(tempos_envio_vendedores, conn)
df

Vamos criar quatro boxplots, correspondendo a cada grupo mostrado no gráfico anterior.

In [ ]:
plt.figure(figsize=(12, 8))

# Cria uma paleta de cores com base no número de grupos de vendedores
palette = sns.color_palette('magma', len(seller_buckets['bucket'].unique()))

# Garante que a ordem dos grupos siga a ordem da variável 'bucket'
ordem_buckets = seller_buckets['bucket'].unique()

# Cria os boxplots com as cores e ajustes visuais
sns.boxplot(
    x='bucket',
    y='delivery_time',
    data=df,
    order=ordem_buckets,
    showfliers=False,   # remove outliers para um gráfico mais limpo
    hue='bucket',
    palette=palette,
    dodge=False
)

plt.title('Tempo de entrega por volume de pedidos do vendedor', fontsize=14)
plt.xlabel('Grupos de vendedores (por número de pedidos)', fontsize=12)
plt.ylabel('Tempo de entrega (dias)', fontsize=12)
plt.legend().remove()
plt.show()


Por algum motivo, quanto maior o vendedor, mais demoradas tendem a ser as entregas.

Um análise futura pode ser a seguinte:
- O que causa esse aumento no tempo de envio?
- Será que os vendedores menores demoram menos entre a aprovação do pedido e a entrega do pacote à transportadora?

<br>

# Conclusão do projeto

Este projeto demonstrou como é possível conduzir uma análise completa de e-commerce utilizando apenas SQL para manipulação e exploração dos dados, com Python aplicado à visualização e storytelling dos resultados. A abordagem evidenciou o poder das CTEs, joins e funções agregadas do SQL em conjunto com as bibliotecas pandas, matplotlib, seaborn e folium para transformar dados brutos em insights acionáveis.


A análise do dataset da Olist revelou padrões claros de comportamento e performance:

  📈 Crescimento constante nas vendas, com picos notáveis em dezembro, impulsionados pelas compras de Natal.

  ⏰ Maior volume de pedidos entre 10h e 16h nos dias úteis, reforçando a importância de campanhas matinais.

  🏙️ Concentração de vendas nas grandes capitais, especialmente São Paulo e Rio de Janeiro, refletindo densidade populacional e poder de compra.

  💸 Pedidos com valor médio em torno de R$160, predominando produtos leves e de ticket baixo.

  🧴 Categorias como “Beleza e Saúde” e “Informática” lideram em volume de vendas e crescimento ao longo do tempo.

  🚚 Diferenças logísticas significativas entre cidades, com prazos de entrega maiores fora do eixo Sudeste.

  ⭐ Predomínio de avaliações positivas, mas com recorrentes reclamações sobre atrasos na entrega.

  💰 Valor do Tempo de Vida do Cliente (LTV) concentrados no Sudeste, indicando regiões prioritárias para retenção e marketing.

  🧾 Maioria dos vendedores é composta por pequenos lojistas, enquanto grandes vendedores, curiosamente, possuem prazos de entrega mais longos.

Além dos resultados, o projeto reforça minha capacidade de:

- Estruturar pipelines analíticos em SQL com CTEs reutilizáveis e lógicas otimizadas;

- Traduzir consultas complexas em visualizações claras e interpretáveis;

- Gerar insights estratégicos orientados a negócio — do comportamento de compra à eficiência operacional.

Em resumo, este estudo combinou raciocínio analítico, domínio técnico e storytelling de dados, transformando o banco da Olist em uma narrativa visual e prática — exatamente o tipo de trabalho que une análise, negócios e impacto real.